# Búsqueda

En esta clase resolveremos un problema claśico, el 8-puzzle, utilizando Breadth-First Search (BrFS).




La clase `Node` representa un search state genérico, que será útil para cualquier problema de búsqueda. 

Cada uno de los estados de búsqueda, tendrá asociado un puntaje almacenado en la propiedad `key`, el cual nos indicará cuando un nodo es mejor (o solo mas prometedor) que otro. En este caso, a menor `key`, mas prometedor es el nodo. 

Tendrá los métodos __gt__ y __lt__ implementan los operadores mayor y menor que, y verifican que si nodo `v` es menor que un nodo `u`, el nodo `v` es mejor.

In [1]:
class Node:
    def __init__(self, search_state, parent=None, action=''):
        self.state = search_state
        self.parent = parent      # Es el estado padre.
        if parent:                # compatibilidad con DFS/BrFS
            self.depth = parent.depth + 1
        else:
            self.depth = 0
        self.action = action    # es el nombre de la accion
        #self.heap_index = 0     
        
    def __repr__(self):
        return self.state.__repr__()

    def trace(self): # reconstruye la solución
        s = ''
        if self.parent:
            s = self.parent.trace()
            s += '-' + self.action + '->'
        s += str(self.state)
        return s

#clase Puzzle

Esta representa el problema. La función mas importante es el método `succ`, que se encarga de la generación de estados sucesores.

In [2]:
import sys
import copy
import numpy as np


class Puzzle:
    goal8 = list(range(9))
    def __init__(self, board=None, blank=-1):
        if not board:
            self.x = 3
            self.size = 9
            self.board = [i for i in range(0, self.size)]
            self.blank = 0
        else:
            self.board = board
            if len(self.board) == 9:
                self.x = 3
                self.size = 9
            else:
                print('puzzle size not supported')
                sys.exit(1)
            if blank == -1:
                self.blank = board.index(0)


    def __hash__(self):
        return hash(tuple(self.board))

    def __eq__(self, other):
        return self.board == other.board

    def __repr__(self):
        def tostr(d):
            if d > 0:
                return "%2d" % (d)
            else:
                return "  "
        s = '\n'
        for i in range(0, self.x):
            s += "|"
            s += "|".join([tostr(d) for d in self.board[i*self.x:i*self.x+self.x]])
            s += "|\n"
        return s

    def successors(self):
        '''
            Crea una lista de tuplas de la forma (estado, accion, costo)
            donde estado es el estado sucesor de self que se genera al ejecutar
            accion (un string) y costo (un numero real) es el costo de accion
        '''
        def create_child(newblank):
            child = copy.deepcopy(self)
            child.blank = newblank
            child.board[child.blank] = 0
            child.board[self.blank] = self.board[newblank]
            return child

        succ = []
        if self.blank > self.x - 1:
            c = create_child(self.blank-self.x)
            succ.append((c, 'down', 1))
        if self.blank % self.x > 0:
            c = create_child(self.blank-1)
            succ.append((c, 'right', 1))
        if self.blank % self.x < self.x - 1:
            c = create_child(self.blank+1)
            succ.append((c, 'left', 1))
        if self.blank < self.size - self.x:
            c = create_child(self.blank+self.x)
            succ.append((c, 'up', 1))
        return succ

    def is_goal(self):
        return Puzzle.goal8 == self.board 

In [3]:
from collections import deque 

class OpenList_BrFS:
    # Inicia una lista vacia
    def __init__(self):
        self.items = deque([])

    # La cantidad de elementos en el heap
    @staticmethod
    def size(self):
        return len(self.items)

    # Retorna el mejor elemento del heap
    def top(self):
        if self.size == 0:
            return None
        else:
            return self.items[0]

    # Retorna el primer elemento y lo elimina
    def extract(self):
        if len(self.items):
            return self.items.popleft()
        else : 
            None

    # Inserta un elemento en el heap
    def insert(self, element):
        self.items.append(element)

    # Verifica si esta vacia
    def is_empty(self):
        return self.size == 0


In [4]:
import time

class BrFS:
    def __init__(self, initial_state):
        self.expansions = 0
        #self.generated = 0 # es la lista de estados generados OPEN U CLOSE
        self.initial_state = initial_state
        self.solution = None
        self.open = OpenList_BrFS()
        self.generated = {}



    def search(self):
        self.start_time = time.process_time()
        
        initial_node = Node(self.initial_state)
        initial_node.g = 0
        self.open.insert(initial_node)      
        self.generated[self.initial_state] = initial_node # para cada estado alguna vez generado, generated almacena el Node object que le corresponde

        while not self.open.is_empty():           # Mientras la OPEN no esté vacia  (ciclo de expansiones) 
            n = self.open.extract()               # extrae n de la open (el con menor key (en este caso profundidad))

            if n.state.is_goal():                 # Si el estado extraido es el estado objetivo (el problema resuelto)
                self.end_time = time.process_time() 
                self.solution = n                 # Define el estado extraido como solución (para reconstruirlo luego)
                return n
            succ = n.state.successors()           # Genera los sucesores
            self.expansions += 1                  # Aumenta en 1 las expansiones
            for child_state, action, cost in succ:            # Por cada uno de los estados sucesores
                child_node = self.generated.get(child_state)  # Si no existe en generados retorna 0 (false)
                path_cost = n.g + cost                        # costo del camino encontrado hasta child_state
                if child_node is None :                       # si el estado no existe en OPEN U CLOSE
                    child_node = Node(child_state, n)         # creamos el nodo de child_state
                    self.generated[child_state] = child_node  # lo agregamos a generados
                    child_node.action = action                # definimos el nombre de la acción del nodo
                    child_node.parent = n                     # definimos el padre
                    child_node.g = path_cost                  # aumentamos el costo del camino con el costo de la accion
                    self.open.insert(child_node)              # agrega el nodo a la OPEN 
        
        self.end_time = time.process_time()  
        print("No solution found")                # Si el ciclo de expansiones termina sin encontrar solución, no existe solución.
        return None

    

In [5]:
problem = Puzzle([4, 7, 5, 0, 8, 2, 6, 1, 3])

s = BrFS(problem) 
result = s.search()
print('Expansiones: %10d' % s.expansions )
print('Generados: %10d' % len(s.generated) )
print('Costo: %10d' % result.g )
print('Tiempo: %10.2f' % (s.end_time-s.start_time) )

print("Solucion*-***************************************")
print(result.trace())

Expansiones:     142540
Generados:     161359
Costo:         25
Tiempo:      10.64
Solucion*-***************************************

| 4| 7| 5|
|  | 8| 2|
| 6| 1| 3|
-down->
|  | 7| 5|
| 4| 8| 2|
| 6| 1| 3|
-left->
| 7|  | 5|
| 4| 8| 2|
| 6| 1| 3|
-left->
| 7| 5|  |
| 4| 8| 2|
| 6| 1| 3|
-up->
| 7| 5| 2|
| 4| 8|  |
| 6| 1| 3|
-right->
| 7| 5| 2|
| 4|  | 8|
| 6| 1| 3|
-down->
| 7|  | 2|
| 4| 5| 8|
| 6| 1| 3|
-right->
|  | 7| 2|
| 4| 5| 8|
| 6| 1| 3|
-up->
| 4| 7| 2|
|  | 5| 8|
| 6| 1| 3|
-up->
| 4| 7| 2|
| 6| 5| 8|
|  | 1| 3|
-left->
| 4| 7| 2|
| 6| 5| 8|
| 1|  | 3|
-left->
| 4| 7| 2|
| 6| 5| 8|
| 1| 3|  |
-down->
| 4| 7| 2|
| 6| 5|  |
| 1| 3| 8|
-right->
| 4| 7| 2|
| 6|  | 5|
| 1| 3| 8|
-right->
| 4| 7| 2|
|  | 6| 5|
| 1| 3| 8|
-up->
| 4| 7| 2|
| 1| 6| 5|
|  | 3| 8|
-left->
| 4| 7| 2|
| 1| 6| 5|
| 3|  | 8|
-down->
| 4| 7| 2|
| 1|  | 5|
| 3| 6| 8|
-down->
| 4|  | 2|
| 1| 7| 5|
| 3| 6| 8|
-right->
|  | 4| 2|
| 1| 7| 5|
| 3| 6| 8|
-up->
| 1| 4| 2|
|  | 7| 5|
| 3| 6| 8|
-up->
| 1| 4| 2|
|

In [6]:
#prblema 3 : 03 0 1 8 7 2 4 5 6 3

problem = Puzzle([0, 1, 8, 7, 2, 4, 5, 6, 3])

s = BrFS(problem) 
result = s.search()
print('Expansiones: %10d' % s.expansions )
print('Generados: %10d' % len(s.generated) )
print('Costo: %10d' % result.g )
print('Tiempo: %10.2f' % (s.end_time-s.start_time) )

print("Solucion*-***************************************")
print(result.trace())

Expansiones:      44479
Generados:      61733
Costo:         20
Tiempo:       3.18
Solucion*-***************************************

|  | 1| 8|
| 7| 2| 4|
| 5| 6| 3|
-left->
| 1|  | 8|
| 7| 2| 4|
| 5| 6| 3|
-up->
| 1| 2| 8|
| 7|  | 4|
| 5| 6| 3|
-left->
| 1| 2| 8|
| 7| 4|  |
| 5| 6| 3|
-down->
| 1| 2|  |
| 7| 4| 8|
| 5| 6| 3|
-right->
| 1|  | 2|
| 7| 4| 8|
| 5| 6| 3|
-up->
| 1| 4| 2|
| 7|  | 8|
| 5| 6| 3|
-up->
| 1| 4| 2|
| 7| 6| 8|
| 5|  | 3|
-right->
| 1| 4| 2|
| 7| 6| 8|
|  | 5| 3|
-down->
| 1| 4| 2|
|  | 6| 8|
| 7| 5| 3|
-left->
| 1| 4| 2|
| 6|  | 8|
| 7| 5| 3|
-up->
| 1| 4| 2|
| 6| 5| 8|
| 7|  | 3|
-left->
| 1| 4| 2|
| 6| 5| 8|
| 7| 3|  |
-down->
| 1| 4| 2|
| 6| 5|  |
| 7| 3| 8|
-right->
| 1| 4| 2|
| 6|  | 5|
| 7| 3| 8|
-up->
| 1| 4| 2|
| 6| 3| 5|
| 7|  | 8|
-right->
| 1| 4| 2|
| 6| 3| 5|
|  | 7| 8|
-down->
| 1| 4| 2|
|  | 3| 5|
| 6| 7| 8|
-left->
| 1| 4| 2|
| 3|  | 5|
| 6| 7| 8|
-down->
| 1|  | 2|
| 3| 4| 5|
| 6| 7| 8|
-right->
|  | 1| 2|
| 3| 4| 5|
| 6| 7| 8|



In [7]:
#prblema 3 : 03 0 1 8 7 2 4 5 6 3

problem = Puzzle([3, 1, 2, 0, 4, 5, 6, 7, 8])

s = BrFS(problem) 
result = s.search()
print('Expansiones: %10d' % s.expansions )
print('Generados: %10d' % len(s.generated) )
print('Costo: %10d' % result.g )
print('Tiempo: %10.2f' % (s.end_time-s.start_time) )

print("Solucion*-***************************************")
print(result.trace())

Expansiones:          1
Generados:          4
Costo:          1
Tiempo:       0.00
Solucion*-***************************************

| 3| 1| 2|
|  | 4| 5|
| 6| 7| 8|
-down->
|  | 1| 2|
| 3| 4| 5|
| 6| 7| 8|



# Preguntas:
- La solución encontrada es la óptima (no existe otra solución con menor costo)? ¿Por que? Demuestrelo.
- Convierta la clase BrFS a la clase DFS (Depth-First Search [Búsqueda en profundidad]) y al algoritmo de Dijkstra. 
- ¿La solución encontrada por DFS es la óptima? ¿Por que? Demuestrelo.